In [6]:
def make_schematic_plot():
    matplotlib.rcParams.update({'font.size': 16})


    # approximate fault locations, sizes, dips
    p1 = np.array([11.15, -15.17, -7.2])
    p2 = np.array([-10.58, 10.88, -8.98])
    p = p1-p2
    laqstrikewidth = np.linalg.norm(p)
    laqdipwidth = .4 * laqstrikewidth

    camstrikewidth = laqstrikewidth
    camdipwidth = .4 * camstrikewidth 

    citstrikewidth = np.array([10])
    citdipwidth = .4 * citstrikewidth

    cit = [-16.8,  25.5, -5.09, citstrikewidth, citdipwidth, - 5 * np.pi/4, 50 * np.pi/180]
    cit2= [-16.8,  25.5, -5.09, citstrikewidth, citdipwidth/2, - 5 * np.pi/4, -50 * np.pi/180]
    cit3= [-16.8+.1,  25.5, -4.09, citstrikewidth, citdipwidth/2, - 5 * np.pi/4, -50 * np.pi/180]
#     cit2= [-16.8,  25.5, -5.09, citstrikewidth, citdipwidth, - 5 * np.pi/4, 50 * np.pi/180]

    cam = [-2.83,  17.3, -8.55, camstrikewidth, camdipwidth, - 5 * np.pi/4, 50 * np.pi/180]
    laq = [ 4.31, -3.64, -6.78, laqstrikewidth, laqdipwidth, - 5 * np.pi/4, 50 * np.pi/180]
    allFaults = [cit, cit2, cit3, cam, laq]

    def makeSchemSurfPoints(x, y, z, xWidth, yWidth, s, d, reps = 2):
        # return points on at outside of fault based on inner point of fault
        __, strikeUnit = faultUnitVectors(s, d, 0       )
        __, dipUnit    = faultUnitVectors(s, d, -np.pi/2)

        points = np.zeros((reps, reps, 3))
        points[:, :] = np.array([x, y, z])

    #     mults = np.array([-.5, .5])
        mults = np.linspace(-.5, .5, reps)
        for i, m1 in enumerate(mults):
            for j, m2 in enumerate(mults):
                points[i, j] += m1 * strikeUnit * xWidth + m2 * dipUnit * yWidth

        xs = points[:, :, 0].ravel()
        ys = points[:, :, 1].ravel()
        zs = points[:, :, 2].ravel()

        return xs, ys, zs




    fig=plt.figure(figsize=(12, 12))
    ax=fig.add_subplot(111,projection='3d')

    for ifault, fault in enumerate(allFaults):
        colors = ['brown', 'brown', 'brown', 'blue', 'green']
        reps = 2
        x, y, z = makeSchemSurfPoints(*fault, reps = reps)
        x=x.reshape((reps, reps))
        y=y.reshape((reps, reps))
        z=z.reshape((reps, reps))
        ax.plot_surface(x, y, z, linewidth=1, edgecolors='k', alpha = .5, color = colors[ifault])

    arrow = [np.array([-20, -20]),
             np.array([0, 10]),
             np.array([0, 0]) ]
    class Arrow3D(FancyArrowPatch):
        # arrows in 3D
        def __init__(self, xs, ys, zs, *args, **kwargs):
            FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
            self._verts3d = xs, ys, zs

        def draw(self, renderer):
            xs3d, ys3d, zs3d = self._verts3d
            xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
            self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
            FancyArrowPatch.draw(self, renderer)
    a = Arrow3D(*arrow, mutation_scale=20, 
                    lw=3, arrowstyle="-|>")
    ax.add_artist(a)
    ax.text3D(arrow[0][1], arrow[1][1], arrow[2][1], 'N')

    arrowUp = [arrow[0],
             np.array([0, 0]),
             np.array([0, 10])]
    a = Arrow3D(*arrowUp, mutation_scale=20, 
                    lw=3, arrowstyle="-|>")
    ax.add_artist(a)
    ax.text3D(arrowUp[0][1], arrowUp[1][1], arrowUp[2][1], 'UP')


    scaleBar = [np.array([-15,  -15]),
                np.array([0 ,  10]),
                np.array([0, 0]) ]
    b = Arrow3D(*scaleBar, mutation_scale=20, lw=3, arrowstyle = '-')
    ax.add_artist(b)
    ax.text3D(scaleBar[0][1]+1, scaleBar[1][1] - 5, scaleBar[2][1], '10 Km')

    labels = ['Cittareale', 'Campotosto', 'Paganica']
    for i, fault in enumerate([cit, cam, laq]
):
        ax.text3D(fault[0], fault[1], fault[2], labels[i], zorder = 10)

    l = ax.get_xlim()
    ax.set_ylim(l)
    ax.set_zlim(l)

    groundrange = np.array(l, l)
    xg, yg = np.meshgrid(groundrange,groundrange)
    zg = np.zeros(xg.shape)
    ax.plot_surface(xg, yg, zg, alpha = .1)

    ax.set_xlabel('East (Km)')
    ax.set_ylabel('North (Km)')
    ax.set_zlabel('Vertical (Km)')


    ax.xaxis.pane.set_edgecolor('black')
    ax.yaxis.pane.set_edgecolor('black')
    ax.zaxis.pane.set_edgecolor('black')
    ax.grid(False)
    plt.tight_layout()
    # ax.view_init(elev = 20, azim = -10)
    if False:
        plt.savefig('figures/schematic.pdf', dpi = 500)
    plt.show()

In [1]:
import numpy as np
import plotly.graph_objs as go 
from plotly.offline import iplot as plot
import sys
import pickle

try:
    sys.path.append('geo_data')
    from load_fault_traces import load_fault_traces
    hypo_faults = pickle.load(open('stored_variables/preliminary_clusters','rb'))
    thing = pickle.load(open('geo_data/traceinfo', 'rb'))
    fault_color_dict = thing['fault_color_dict']
    trace_to_label = thing['trace_to_label']
    trace_to_label['Cittareale']=None # Don't plot trace color for Cittareale fault, not relevant here. 
except:
    pass
# from load_fault_traces import load_fault_traces
# from importlib import reload


# # def make_fault_trace_plot():
# fault_color_dict = {
#                     'Paganica':'#32a852',  
#                     'Campotosto':'#4f4fff', 
#                     'Cittareale':'#c74c00',
#                     'Mt. San Franco':'#ff8c00',
#                     # bellow address faults which are only present in fault traces
#                     'Mt. Stabiata':'#a900c7',
#                     'Mt. Castellano-Colle Enzano':'#f2ff00',
#                     #
#                     'Surface rupture':'#ff0000',
#                     'Noise'     :'#E6E6E6',
#                     }#'#757575'}

# # Convert from names in the fault trace file to names I want in the legend. 
# # trace_to_label = {'M.Stabbiata':'Mt. Stabiata', # This line is shared with area_plot.py
# #                   'M.S.Franco':'Mt. San Franco',
# #                   'Aternosys.2':'Paganica',
# #                   'Aternosys.6':'Paganica',
# #                   'Aternosys.7':'Paganica',
# #                   'Aternosys.8':'Paganica',
# #                   'Campotosto':'Campotosto',
# #                   'Cittareale':'Cittareale',
# #                   'Mt.Castellano':'Mt. Castellano-Colle Enzano',
# #                   'C.lleEnzano':'Castellano-Colle Enzano',
# #                   }
# trace_to_label = {'M.Stabbiata':'Mt. Stabiata', # This line is shared with area_plot.py # 0901
#                   'M.S.Franco':'Mt. San Franco',
#                   'Aternosys.2':'Paganica',
#                   'Aternosys.6':'Paganica',
#                   'Aternosys.7':'Paganica',
#                   'Aternosys.8':'Paganica',
#                   'Campotosto':'Campotosto',
#                   'Cittareale':'Cittareale',
#                   'Mt.Castellano':'Mt. Castellano-Colle Enzano',
#                   'C.lleEnzano':'Mt. Castellano-Colle Enzano',
#                   }

def add3dfaulttraces(latCent, lonCent):
    faultPlot = []
    mf, sr = load_fault_traces(mffile = 'geo_data/Mapped_Faults', srfile = 'geo_data/Surface_Ruptures')
    # 
    for ifault, fault in enumerate(mf):
        yf, xf = ChangeAxis(latCent, lonCent, fault['lats'], fault['lons'])
        zf = fault['elv']
        fault_name = trace_to_label.get(fault['name'],'not present') # This line is shared with area_plot.py
        faultPlot += [go.Scatter3d(
            x=xf,
            y=yf,
            z=zf,
            name = 'thing',
            mode = 'lines',
            line=dict(
                color=fault_color_dict.get(fault_name, '#612f2f'), # This line is shared with area_plot.py
                width=6) )
                     ]
    for ifault, fault in enumerate(sr):
        yf, xf = ChangeAxis(latCent, lonCent, fault['lats'], fault['lons'])
        zf = np.array(fault['elv']) - 200 # Subtract a bit so the surface ruptures don't cover up other faults
        faultPlot += [go.Scatter3d(
            x=xf,
            y=yf,
            z=zf,
            name = 'thing',
            mode = 'lines',
            line=dict(
                color='rgb(255, 0, 0)',
                width=20) )
                     ]     
    return faultPlot 



In [ ]:
def add3dscalebar():
    textFontDict = {'color':'red', 'size':18}
    ### north arrow
#         xNA = 6.5e3; yNA = 10e3; zNA = -14e3
    xR = 10e3
    xNA = np.amax(xR-3e3)
    yNA = 28e3#np.amax(self.y)
    zNA = -14e3#np.amin(self.z)
    yBase = yNA - 5e3
    northLine = go.Scatter3d(
        x=[xNA, xNA], y=[yBase, yNA], z=[zNA, zNA],
        text = np.array(['', 'N']),
        mode='lines+text',#Change?
        line  =dict(width = 5, color = '#7f7f7f'),
        textposition = 'middle left',
        textfont = textFontDict
    )

    northCone = dict(
        type = 'cone',
        colorscale = 'Greys',
        showscale = False,
        x = [xNA], y = [yNA], z = [zNA],
        u = [0], v = [2e3], w = [0]
        )

    upLine = go.Scatter3d(
        x=[xNA, xNA], y=[yBase, yBase], z=[zNA, zNA+5e3],
        text = np.array(['', 'UP']),
        mode='lines+text',#Change?
        line  =dict(width = 5, color = '#7f7f7f'),
        textposition = 'middle left',
        textfont = textFontDict
    )

    upCone = dict(
        type = 'cone',
        colorscale = 'Greys',
        showscale = False,
        x = [xNA], y = [yBase], z = [zNA+5e3],
        u = [0], v = [0], w = [2e3]
        )
    ###

    ###  scale bar      
    xN = np.array([xR, xR, xR])
    yN = np.array([-10e3, -5e3, 0])+yNA
    zN = np.array([zNA, zNA, zNA])
    scaleBar = go.Scatter3d(
        x=xN,
        y=yN,
        z=zN,
        text = np.array(['', '10 km', '']),
        mode='lines+text',
        line  =dict(width = 15, color = '#7f7f7f'),
        textposition='middle right',
        textfont = textFontDict
    )
    
    return [scaleBar, northLine, northCone, upLine, upCone]
    
def add3dlayout():
    layout = go.Layout(
        showlegend=False,
        autosize=False,
        width=1000,
        height=1000,
        scene=dict(
            camera=dict(eye=dict(x=1.75, y=-0.7, z= 0.75) ),
            aspectmode = 'data',
            xaxis = dict(showticklabels=False, title=''),
            yaxis = dict(showticklabels=False, title=''),
            zaxis = dict(showticklabels=False, title='')
                  )  
        )
    return layout

def add3dclusters(x, y, z, labels, lineWidthR=.1, pointSize = 1):
    
        scatterPlot = [go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=pointSize,
            color=labels,
            colorscale='Jet',
            line=dict(
                    color='rgb(0,0,0)',
                    width=lineWidthR 
                ),
            opacity=1) )]
        
        return scatterPlot
    
def add3dsurface(interpS, interpD, interpX, interpY, interpZ):
    points2D = np.vstack([interpS, interpD]).T 
    tri = Delaunay(points2D)
    surfacePlot = plotly_trisurf(interpX, interpY, interpZ, 
                                tri.simplices, 
#                                  colormap = cm.plasma,
                                 colormap = cm.viridis,
                                 # ListedColormap(np.array([[90, 120, 91]])/256), # cm.cubehelix, 
#                                  colormap = ListedColormap(np.array([[ 0, 255, 68]])/256), 
#                                  colormap = ListedColormap(np.array([[ 255, 183, 0]])/256), 
#                                  colormap = ListedColormap(np.array([[ ff0000]])/256), 
                                 plot_edges = True)
    return surfacePlot

In [ ]:
# Functions which determine what colors to make each cluster
def labels_to_colors(labels, fault_color_dict, color_option = 2):
    colors = np.zeros(labels.shape, dtype = object)
    if color_option == 1:
        un_labels, counts = np.unique(sp.labels, return_counts=True)
        labels123 = un_labels[np.argsort(counts)][::-1] # get unique labels sorted from greatest to least common
        colors[labels == labels123[0]] = fault_color_dict['Paganica']
        colors[labels == labels123[1]] = fault_color_dict['Campotosto']
        colors[labels == labels123[2]] = fault_color_dict['Cittareale']
    elif color_option == 2:
        for ilabel, label in enumerate(np.unique(labels)):
            fault_name, color_db = get_cluster_color(labels, label, hypo_faults, fault_color_dict)
            this = labels == label
            if color_db is None:
                color_db = label
            colors[this] = color_db
    elif color_option == 3:         
        for ilabel, label in enumerate(np.unique(labels)):
            fault_name, color_db = get_cluster_color(labels, label, hypo_faults, fault_color_dict)
            if fault_name != 'Paganica':
                color_db = 'red'
            this = labels == label
            if color_db is None:
                color_db = label
            colors[this] = color_db
    elif color_option == 4: 
        return labels
    return colors
        
def get_cluster_color(labels, label, hypo_faults, fault_color_dict):
    similarity = np.zeros(len(hypo_faults)) # similarity between currently observed cluster and each previously named cluster. 
    name = np.zeros(len(hypo_faults), dtype = object)
    for ikey, key in enumerate(hypo_faults.keys()):
        similarity[ikey] = (hypo_faults[key] * (labels == label)).sum()/(hypo_faults[key].sum()) # Similarity is basically normalized zero lag cross correlation between the boolean aray of this cluster and each previously named cluster
        if similarity[ikey] <= 0:
            name[ikey] = None
        elif similarity[ikey] > 0:
            name[ikey] = key
    if (similarity>0).any():
        fault_name = name[similarity == similarity.max()][0] 
    else:
        fault_name = 'not present'
    color_db    = fault_color_dict.get(fault_name, None)
    return fault_name, color_db

In [16]:
def plot3dclusters(x, y, z, labels, latCent, lonCent, color_option,
                   interpX = None, interpY = None, interpZ = None, interpS = None, interpD = None,
                   pointSize=4, lineWidthR = 1/10, save = False, name = 'to_delete', 
                   faulttraces3d = False, faultcolors = True):
#     colors = labels
    if faultcolors: 
        colors = labels_to_colors(labels, fault_color_dict, color_option = color_option); #print('Trying my fancy color scheme')
    else:
        colors = ['#000000' for i in range(len(x))]
        lineWidthR = 0; 
    scatterPlot = add3dclusters(x = x, y = y, z = z, labels = colors, pointSize = pointSize, lineWidthR = lineWidthR)
    scatterPlot += add3dscalebar()
    if faulttraces3d:
        try:
            scatterPlot += add3dfaulttraces(latCent, lonCent)
        except:
            pass
    if interpX is not None:
        scatterPlot += add3dsurface(interpS, interpD, interpX, interpY, interpZ)
    layout = add3dlayout()
    fig2 = go.Figure(data=scatterPlot, layout=layout)
    plot(fig2)
    
    if name != 'to_delete':
        print('No longer using plotly online: will not save online! turn the save feature off. ')
    return fig2

In [12]:
def add_legend_plotly(fig=None):
    try:
        thing = pickle.load(open('geo_data/traceinfo', 'rb'))
        fault_color_dict = thing['fault_color_dict']
        trace_to_label = thing['trace_to_label']

        if fig is None:
            fig = go.Figure()
        for fault_name in fault_color_dict:
            fig.add_trace(go.Scatter3d(
                x = [0], y = [0], z = [0], name = fault_name, marker = {'color': fault_color_dict[fault_name]}
            ))
    #     import ipython; ipython.embed()
    #     fig.update_layout(showlegend=True)
        fig.show()


        fig.write_image('figures/legend.jpg', scale = 3)
        clear_output()
        im = PIL.Image.open('figures/legend.jpg')
        wid, hei = im.size
        im_crop = im.crop((.65*wid, .2 * hei, .99 * wid, .6 * hei))
        im_crop.save('figures/legend.jpg', quality=100)
        im = PIL.Image.open('figures/legend.jpg')
        im
        return im
    except:
        return None 

In [ ]:
def tri_indices(simplices):
    # Taken from plotly.com. 
    #simplices is a numpy array defining the simplices of the triangularization
    #returns the lists of indices i, j, k

    return ([triplet[c] for triplet in simplices] for c in range(3))

def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, plot_edges=None):
    # Taken from plotly website
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data

    points3D=np.vstack((x,y,z)).T
    tri_vertices=list(map(lambda index: points3D[index], simplices))# vertices of the surface triangles     
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]
    I,J,K=tri_indices(simplices)

    triangles=go.Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor,
#                     color='rgba(39, 135, 32,0.6)',
                     opacity=.6,
                     i=I,
                     j=J,
                     k=K
                    )

    if plot_edges is None:# the triangle sides are not plotted 
        return [triangles]
    else:
#         set_trace()

        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None]   for T in tri_vertices]  for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]

        #define the lines to be plotted
        lines=go.Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=dict(color= 'rgb(0,0,0)', width=1.5)
               )
        return [triangles, lines]